In [1]:
# Specify the range of wavelengths as a tuple (min, max)
wavelength_range = (2.3e-06, 2.32e-06)

resolution_str = 'High-Res'

# Load and sort the wavelengths from the file to ensure they are in order
with open(f'DATA/{resolution_str}/Full-Set/Wavelengths.txt', 'r') as f:
    wavelengths_list = sorted(float(line.strip()) for line in f)

# Find the closest wavelengths in the list to the specified range
min_wavelength = min(wavelengths_list, key=lambda x: abs(x - wavelength_range[0]))
max_wavelength = min(wavelengths_list, key=lambda x: abs(x - wavelength_range[1]))

# Get the indices of the closest matches
min_index = wavelengths_list.index(min_wavelength)
max_index = wavelengths_list.index(max_wavelength)

# Calculate the number of values between the two indices
values_between = max_index - min_index

print(f"Using wavelengths between {min_wavelength} and {max_wavelength}")
print(f"Number of wavelength values between {min_wavelength} and {max_wavelength}: {values_between}")

species = ['CH4', 'CO', 'CO2', 'H2O', 'H2S', 'HCN', 'NH3']

collected_wavelengths = []

first_loop = True
for specie in species:
    input_file_path = f'DATA/{resolution_str}/Full-Set/opac{specie}.dat'
    output_file_path = f'DATA/{resolution_str}/opac{specie}.dat'

    # Specify the wavelengths you are interested in as a list to determine the range
    interested_wavelengths = [min_wavelength, max_wavelength]

    # Process the file to find the range
    start_line = None
    end_line = None

    with open(input_file_path, 'r') as file:
        line_counter = 0
        for line in file:
            line_counter += 1  # Increment line counter for each line

            if line_counter > 2:  # Skip the first two lines which are headers
                parts = line.strip().split()
                if parts:
                    try:
                        wavelength = float(parts[0])
                        
                        if (min_wavelength <= wavelength <= max_wavelength) and first_loop:
                            collected_wavelengths.append(wavelength)
                        if wavelength == interested_wavelengths[0]:
                            start_line = line_counter
                        elif wavelength == interested_wavelengths[1]:
                            end_line = line_counter
                            break  # Stop the loop once the end line is found
                    except ValueError:
                        print(f"Could not convert to float: {parts[0]}")
                        

    # Check if both start and end lines were found
    if start_line is None or end_line is None:
        print("Start or end wavelength not found in the file.")
    else:
        # Write the lines from start_line to one before end_line to the output file
        with open(input_file_path, 'r') as file, open(output_file_path, 'w') as outfile:
            for i, line in enumerate(file, 1):
                if i == 1 or i == 2 or start_line <= i < end_line:
                    outfile.write(line)

        print(f'Filtered data saved to {output_file_path}')
    
    first_loop = False

# Write the collected wavelengths to a new file, excluding the last one
with open('DATA/' + resolution_str + '/Wavelengths.txt', 'w') as f:
    for wavelength in collected_wavelengths[:-1]:  # Exclude the last wavelength
        f.write(f"{wavelength}\n")

print('Wavelengths.txt has been created.')


Using wavelengths between 2.299999038e-06 and 2.320003484e-06
Number of wavelength values between 2.299999038e-06 and 2.320003484e-06: 2165
Filtered data saved to DATA/High-Res/opacCH4.dat
Filtered data saved to DATA/High-Res/opacCO.dat
Filtered data saved to DATA/High-Res/opacCO2.dat
Filtered data saved to DATA/High-Res/opacH2O.dat
Filtered data saved to DATA/High-Res/opacH2S.dat
Filtered data saved to DATA/High-Res/opacHCN.dat
Filtered data saved to DATA/High-Res/opacNH3.dat
Wavelengths.txt has been created.


In [2]:
input_file_path = f'DATA/{resolution_str}/Full-Set/opacCIA.dat'
output_file_path = f'DATA/{resolution_str}/opacCIA.dat'

# Process the file to filter out rows with the first column value outside the specified range
with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
    line_counter = 0  # Counter to track line numbers
    for line in infile:
        line_counter += 1
        parts = line.strip().split()

        # Always write the first two lines and lines with single values
        if line_counter <= 2 or len(parts) == 1:
            outfile.write(line)
        elif len(parts) > 1:
            try:
                wavelength = float(parts[0])
                if min_wavelength <= wavelength < max_wavelength:  # Exclude the upper bound
                    outfile.write(line)
            except ValueError:
                print(f"Could not convert to float: {parts[0]}")
                
print(f'Filtered data saved to {output_file_path}')

Filtered data saved to DATA/High-Res/opacCIA.dat
